# Text Processing

In [1]:
import numpy as np
import pandas as pd
from pymystem3 import Mystem
import re
import transformers
import nltk
from tqdm import notebook
import torch
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForMaskedLM

## Данные

In [2]:
file_path = 'chd — 100.xlsx'

In [3]:
data = pd.read_excel(file_path)
data.head(1)

,admittion,department,discharge,sex,height,weight,BMI,BSA,birth,Операции (все в ИБ),...,Количество выб. из ОРИТ,К.дней в ОРИТ (всего),"ИВЛ, час. в ОРИТ (суммарно)",Инф. осложнения в ОРИТ,Назн. преп. в ОРИТ,target,Непосред. причина смерти,ЭхоКГ (Из Эпикр. вып.),ЭКГ (Из Эпикр. вып.),Назначения при выписке
0,2016-12-12,ehn,2017-01-10,m,76,9.7,111.27,0.46,02.01.2016,12.12.2016: (Откр./ИК) Перевязка ранее наложен...,...,2,20,252,01.01.2017: пневмония,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №10...",recovery,NaN,"ЭхоКГ ВПС (02.12.2016 14:37:22, врач Неталиева...",ЭКГ (02.12.2016 16:43:25)\nРитм сердца синусов...,


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   admittion                                        99 non-null     datetime64[ns]
 1   department                                       99 non-null     object        
 2   discharge                                        99 non-null     datetime64[ns]
 3   sex                                              99 non-null     object        
 4   height                                           99 non-null     int64         
 5   weight                                           99 non-null     float64       
 6   BMI                                              99 non-null     float64       
 7   BSA                                              99 non-null     float64       
 8   birth                                     

In [6]:
data['department'].value_counts()

ehn     46
onik    32
rhn     21
Name: department, dtype: int64

Для теста возьмем только один столбец - диагноз

In [7]:
diagnosis = data['Диагноз']

In [8]:
diagnosis[1]

'двойное отхождение аорты и легочной артерии от правого желудочка, подаортальный дефект межжелудочковой перегородки, комбинированный стеноз легочной артерии, умеренные сужения устьев правой и левой  легочных артерий; ОАП, НК 0-1 ст, артериальная гипоксемия, состояние после ТЛБВП 15.08.2016, клапана ЛА, с-м Дауна, с-м мышечной гипотонии, гипертензионно-гидроцефальный с-м; 18.01.2017 - ОПЕРАЦИЯ: радикальная коррекция двойного отхождения магистральных сосудов от правого желудочка с пластикой выводного отдела правого желудочка и ствола легочной артерии ксеноперикардиальной заплатой; перевязка открытого артериального протока; в условиях ИК, гипотеремии, НК 2а ст'

## Пример применения TF-IDF на столбце "Диагноз"

### Гипотетический способ отчистки конкретных элементов текста (например даты)

In [63]:
nDAY = r'(?:[0-3]?\d)'
nMNTH = r'(?:11|12|10|0?[1-9])' 
nYR = r'(?:(?:19|20)\d\d)'
nDELIM = r'(?:[\/\-\._])?'

In [64]:
re.sub(f'(?:{nDAY}{nDELIM}{nMNTH}{nDELIM}{nYR})', ' ', diagnosis[1])

'Атрезия легочной артерии 1 тип ; Перимембранозный приточный дефект межжелудочковой перегороки (некоммитированный); Открытое овальное окно. Стеноз устья левой легочной артерии, умеренная гипоплазия левой легочной артерии; Дискордантное отхождение аорты от правого желудочка; Состояние после операции ( ) наложение модифицированного подключично-легочного анастомоза по Blalock справа; Большая аорто-легочная коллатеральная артерия к правому легкому; Операция ( ) перевязка ранее наложенного анастомоза по Blalock справа; реконструкция путей оттока из правого желудочка кондуитом из яремной вены быка, в условиях ИК и гипотермии; Недостаточность кровообращения 2-б степени'

### Лемментизация

In [9]:
m = Mystem()

# Леммантизирует текст
def lemmatize(text):
    return "".join(m.lemmatize(text))

In [10]:
lemmatize(diagnosis[1])

'двойной отхождение аорта и легочный артерия от правый желудочек, подаортальный дефект межжелудочковый перегородка, комбинированный стеноз легочный артерия, умеренный сужение устье правый и левый  легочный артерия; оап, НК 0-1 ст, артериальный гипоксемия, состояние после ТЛБВП 15.08.2016, клапан ЛА, с-м даун, с-м мышечный гипотония, гипертензионный-гидроцефальный с-м; 18.01.2017 - операция: радикальный коррекция двойной отхождение магистральный сосуд от правый желудочек с пластика выводной отдел правый желудочек и ствол легочный артерия ксеноперикардиальный заплата; перевязка открытый артериальный проток; в условие ик, гипотеремия, НК 2а ст\n'

### Отчистка текста

In [11]:
# Чистит текст от всего, кроме русских и английских букв
def clear_text(text):
    cleaned = re.sub(r'[^а-яА-Яa-zA-ZёЁ ]', ' ', text)
    cleaned = cleaned.split()
    return ' '.join(cleaned)

In [12]:
clear_text(lemmatize(diagnosis[1]))

'двойной отхождение аорта и легочный артерия от правый желудочек подаортальный дефект межжелудочковый перегородка комбинированный стеноз легочный артерия умеренный сужение устье правый и левый легочный артерия оап НК ст артериальный гипоксемия состояние после ТЛБВП клапан ЛА с м даун с м мышечный гипотония гипертензионный гидроцефальный с м операция радикальный коррекция двойной отхождение магистральный сосуд от правый желудочек с пластика выводной отдел правый желудочек и ствол легочный артерия ксеноперикардиальный заплата перевязка открытый артериальный проток в условие ик гипотеремия НК а ст'

### TF-IDF

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Стивен\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stopwords = set(nltk_stopwords.words('russian'))

In [15]:
%%time

corpus = diagnosis.apply(lambda x: clear_text(lemmatize(x)))

CPU times: total: 156 ms
Wall time: 1min 17s


In [33]:
tf_idf = TfidfVectorizer(stop_words=stopwords).fit(corpus)
tf_idf_train = tf_idf.transform(corpus)

In [34]:
tf_idf_train.toarray().shape

(99, 481)

In [38]:
tf_idf.vocabulary_

{'атрезия': 40,
 'легочный': 203,
 'артерия': 34,
 'тип': 419,
 'перимембранозный': 286,
 'приточный': 331,
 'дефект': 120,
 'межжелудочковый': 212,
 'перегорока': 281,
 'некоммитированный': 236,
 'открывать': 269,
 'овальный': 254,
 'окно': 259,
 'стеноз': 405,
 'устье': 446,
 'левый': 201,
 'умеренный': 443,
 'гипоплазия': 94,
 'дискордантный': 125,
 'отхождение': 272,
 'аорта': 26,
 'правый': 326,
 'желудочек': 140,
 'состояние': 394,
 'операция': 261,
 'наложение': 228,
 'модифицированный': 221,
 'подключичный': 305,
 'анастомоз': 18,
 'blalock': 1,
 'справа': 399,
 'большой': 50,
 'аорто': 28,
 'коллатеральный': 171,
 'легкий': 202,
 'перевязка': 278,
 'ранее': 346,
 'налагать': 227,
 'реконструкция': 364,
 'путь': 340,
 'отток': 271,
 'кондуит': 179,
 'яремный': 480,
 'вена': 59,
 'бык': 54,
 'условие': 445,
 'ик': 149,
 'гипотермия': 96,
 'недостаточность': 235,
 'кровообращение': 190,
 'степень': 409,
 'двойной': 111,
 'подаортальный': 301,
 'перегородка': 279,
 'комбинированны

In [63]:
tf_idf_vocab = pd.DataFrame([[i, j] for i, j in tf_idf.vocabulary_.items()], columns=['word', 'index'])
tf_idf_vocab = tf_idf_vocab.set_index('index')
tf_idf_vocab = tf_idf_vocab.sort_index()
tf_idf_vocab.head(10)

,word
index,
0,alfieri
1,blalock
2,by
3,carbomedics
4,gore
5,ii
6,inversus
7,mastard
8,muller


## RuBert

In [125]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

model = AutoModelForMaskedLM.from_pretrained("sberbank-ai/ruBert-base")

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [126]:
tokenized = corpus.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [135]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

194

In [138]:
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
padded[0]

array([   101,    116,   2991,  74639, 113404,    666,    107,  30545,
        12866,    378,  31338,    390,   1996,  89679,  20900,    378,
        71738,   5635,  62761,    701,  10429,    391,    378,  39381,
          657,   3526,  91552,    378,  73147,    667,    378,  36986,
          396,  30545,  12866,    378,  31338,    390,  61896,  52065,
         6056,  91706,  20900,    378,  73147,  11894, 113404,  91645,
         1277,  12866,    378,  22389,   2154,  20900,    378,  31338,
          390,    700,  80548,    755,    378, 113404,    666,    114,
         4984,   2160,  12020,   5762,    934,  11803,  45165,   9309,
        41156,    667,    378,  30545,  12866,    378,   3904,   2968,
         4211,  13474,    108,    385,    106,  11894,  99540,    667,
          378,  31338,    922,    667,    378,  31053,  69367,   5917,
        11803,  71738,   5635,   8838,   3624,  33988,    378,  39381,
          657,  83047,    660,  30545,  12866,    378,  31338,    390,
      

In [149]:
batch_size = 10
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    print(f'Stage {i}')
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/10 [00:00<?, ?it/s]

Stage 0
Stage 1
Stage 2
Stage 3
Stage 4
Stage 5
Stage 6
Stage 7
Stage 8
Stage 9


In [151]:
features = np.concatenate(embeddings)

In [154]:
features.shape

(100, 120138)

Получилось 120000 признаков. **Надо использовать маленькую библиотеку слов, желательно из медицинских терминов.**

## Тест модели

In [19]:
X, y = tf_idf_train, data['department']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [57]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=100)
model = model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.102631
0:	learn: 0.9883221	test: 0.9941993	best: 0.9941993 (0)	total: 4.57ms	remaining: 4.57s
100:	learn: 0.0496310	test: 0.0783668	best: 0.0783668 (100)	total: 417ms	remaining: 3.71s
200:	learn: 0.0224152	test: 0.0392966	best: 0.0392966 (200)	total: 823ms	remaining: 3.27s
300:	learn: 0.0143153	test: 0.0266355	best: 0.0266355 (300)	total: 1.24s	remaining: 2.88s
400:	learn: 0.0102980	test: 0.0199925	best: 0.0199925 (400)	total: 1.65s	remaining: 2.46s
500:	learn: 0.0079492	test: 0.0160101	best: 0.0160101 (500)	total: 2.06s	remaining: 2.05s
600:	learn: 0.0064301	test: 0.0132768	best: 0.0132768 (600)	total: 2.46s	remaining: 1.64s
700:	learn: 0.0054365	test: 0.0114769	best: 0.0114769 (700)	total: 2.88s	remaining: 1.23s
800:	learn: 0.0046577	test: 0.0100422	best: 0.0100422 (800)	total: 3.29s	remaining: 817ms
900:	learn: 0.0040926	test: 0.0089699	best: 0.0089699 (900)	total: 3.69s	remaining: 406ms
999:	learn: 0.0036411	test: 0.0081128	best: 0.0081128 (999)	total: 4.09s	

In [68]:
tf_idf_vocab['importance'] = model.get_feature_importance()
tf_idf_vocab_sorted = tf_idf_vocab.sort_values(by='importance', ascending=False)
tf_idf_vocab_sorted.head(10)

,word,importance
index,,
244,нк,53.674759
400,ст,20.447456
409,степень,11.298997
167,клапан,0.809891
445,условие,0.695534
235,недостаточность,0.640112
437,трикуспидальный,0.637504
261,операция,0.630894
353,расширение,0.623926


In [22]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         ehn       1.00      1.00      1.00         9
        onik       1.00      1.00      1.00         7
         rhn       1.00      1.00      1.00         4

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



In [28]:
cat_score = cross_val_score(model, X, y)
cat_score

Learning rate set to 0.069519
0:	learn: 1.0510539	total: 4.58ms	remaining: 4.57s
100:	learn: 0.0693437	total: 389ms	remaining: 3.46s
200:	learn: 0.0327471	total: 799ms	remaining: 3.17s
300:	learn: 0.0206060	total: 1.23s	remaining: 2.86s
400:	learn: 0.0147074	total: 1.65s	remaining: 2.46s
500:	learn: 0.0114347	total: 2.05s	remaining: 2.04s
600:	learn: 0.0093283	total: 2.44s	remaining: 1.62s
700:	learn: 0.0079320	total: 2.83s	remaining: 1.21s
800:	learn: 0.0068294	total: 3.22s	remaining: 800ms
900:	learn: 0.0059665	total: 3.63s	remaining: 399ms
999:	learn: 0.0053608	total: 4.01s	remaining: 0us
Learning rate set to 0.069519
0:	learn: 1.0194949	total: 4.55ms	remaining: 4.55s
100:	learn: 0.0720492	total: 394ms	remaining: 3.51s
200:	learn: 0.0341635	total: 790ms	remaining: 3.14s
300:	learn: 0.0218396	total: 1.21s	remaining: 2.8s
400:	learn: 0.0158377	total: 1.61s	remaining: 2.41s
500:	learn: 0.0120536	total: 2.02s	remaining: 2.01s
600:	learn: 0.0097989	total: 2.42s	remaining: 1.61s
700:	lear

array([1. , 1. , 0.9, 1. , 1. ])

In [29]:
cat_score.mean()

0.9800000000000001

In [23]:
dummy_model = DummyClassifier().fit(X_train, y_train)
print(classification_report(y_test, dummy_model.predict(X_test)))

              precision    recall  f1-score   support

         ehn       0.45      1.00      0.62         9
        onik       0.00      0.00      0.00         7
         rhn       0.00      0.00      0.00         4

    accuracy                           0.45        20
   macro avg       0.15      0.33      0.21        20
weighted avg       0.20      0.45      0.28        20



C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
cross_val_score(DummyClassifier(), X, y)

array([0.5       , 0.45      , 0.45      , 0.45      , 0.47368421])